In [1]:
# imports
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import os

In [2]:
#users = interactions_emb
interactions = pd.read_csv("embeddings/users_filtered_final.csv") #document with user interactions
interactions.head()

,User,ID,Interactions_emb
0,U244,N17157 N38621 N35022 N50578 N264 N9120 N23907 ...,"[-0.005149974951877837, -0.013250857458654631,..."
1,U68369,N19381 N54536,"[0.0025621717686590273, 0.004183989018201828, ..."
2,U50236,N4020 N44292 N50292 N40772 N57737 N33969 N4054...,"[-0.010138329240492436, -0.01179651383115145, ..."
3,U77060,N23105 N41375,"[-0.005568941123783588, -0.025914330035448074,..."
4,U5596,N459 N56253 N62931 N55846 N29849 N45729 N62834...,"[-0.012533644353970886, -0.011675744312297967,..."


In [3]:
#load the data with news articles
news = pd.read_csv("embeddings/news_emb_final.csv") #document with news content
news.head()

,ID,Category,SubCategory,Content,Content_emb
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","[0.005885085556656122, -0.007782096974551678, ..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat These seemingly ...,"[-0.004876355174928904, -0.007969613187015057,..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,"[-0.02760046347975731, -0.013719998300075531, ..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"[-0.0297758337110281, -0.014837449416518211, 0..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","[0.005073545966297388, 0.004160495940595865, 0..."


In [4]:
def list_ids_in_folder(folder_path):
    ids = set()
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # Extracting the ID from the filename
            file_id = filename.split("_")[0][1:]
            ids.add(file_id)
    
    return list(ids)

In [5]:
folder_path = 'content_recommendations'
users_list = list_ids_in_folder(folder_path)
users_list = ['U' + num for num in users_list]
len(users_list)

3009

Category correlation

In [6]:
def category_correlation(method, n_recommend):
    
    all_pearson = []
    all_pvalue = []
    
    if method == "content":
        folder_path = 'content_recommendations'
    
    users_list = list_ids_in_folder(folder_path)
    users_list = ['U' + num for num in users_list]
    
    for user_id in users_list:
        if method == "content":
            item_path = "content_recommendations/" + user_id + "_content.csv"
        read_articles = interactions[interactions['User'] == user_id]['ID'].str.split().explode().tolist()
        user_news_df = news[news['ID'].isin(read_articles)]
        categ_counts_user = user_news_df['Category'].value_counts()
            
        recommendation_df = pd.read_csv(item_path)
        recpmmendations = recommendation_df.head(n_recommend)
        rec_list = recpmmendations['ID'].tolist() 
        user_news_rec = news[news['ID'].isin(rec_list)]
        recom_counts_user = user_news_rec['Category'].value_counts()
            
        merged_df = pd.concat([recom_counts_user, categ_counts_user], axis=1, sort=False).fillna(0)
        merged_df.columns = ['recommend', 'profiles']
        
        try:
            correlation, p_value = pearsonr(merged_df['recommend'], merged_df['profiles'])
            all_pearson.append(correlation)
            all_pvalue.append(p_value)
        except ValueError as e:
            print("An error occurred:", e)
            print("Skipping correlation calculation due to insufficient data.")
        
    print("Pearson correlation coefficient:", np.nanmean(all_pearson))
    print("P-value:", np.nanmean(all_pvalue))

In [12]:
category_correlation("content", 20)

An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation cal

Subcategory correlation 

In [9]:
def subcategory_correlation(method, n_recommend):
    
    all_pearson = []
    all_pvalue = []
    
    if method == "content":
        folder_path = 'content_recommendations'
    
    users_list = list_ids_in_folder(folder_path)
    users_list = ['U' + num for num in users_list]
    
    for user_id in users_list:
        if method == "content":
            item_path = "content_recommendations/" + user_id + "_content.csv"
        read_articles = interactions[interactions['User'] == user_id]['ID'].str.split().explode().tolist()
        user_news_df = news[news['ID'].isin(read_articles)]
        categ_counts_user = user_news_df['SubCategory'].value_counts()
            
        recommendation_df = pd.read_csv(item_path)
        recpmmendations = recommendation_df.head(n_recommend)
        rec_list = recpmmendations['ID'].tolist() 
        user_news_rec = news[news['ID'].isin(rec_list)]
        recom_counts_user = user_news_rec['SubCategory'].value_counts()
            
        merged_df = pd.concat([recom_counts_user, categ_counts_user], axis=1, sort=False).fillna(0)
        merged_df.columns = ['recommend', 'profiles']
        
        try:
            correlation, p_value = pearsonr(merged_df['recommend'], merged_df['profiles'])
            all_pearson.append(correlation)
            all_pvalue.append(p_value)
        except ValueError as e:
            print("An error occurred:", e)
            print("Skipping correlation calculation due to insufficient data.")
        
    print("Pearson correlation coefficient:", np.nanmean(all_pearson))
    print("P-value:", np.nanmean(all_pvalue))

In [10]:
subcategory_correlation("content", 20)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
Pearson correlation coefficient: 0.3231558348241033
P-value: 0.27230192557393856


Sentiment correlation

In [16]:
news = pd.read_csv("sentiment_analysis/sentiment_final.csv") #document with user interactions
news = news.iloc[:, 1:] 
news = news.drop(['Content_emb'], axis=1)
news.head()

,ID,Category,SubCategory,Content,Sentiment
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...",Positive
1,N19639,health,weightloss,50 Worst Habits For Belly Fat These seemingly ...,Negative
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Negative
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,Neutral
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...",Positive


In [13]:
def analyze_sentiments(news_df, behaviors_df):
    # Create a dictionary to store sentiments for each ID
    id_sentiments = {}

    # Iterate through rows in the news dataframe
    for index, row in news_df.iterrows():
        # Check if the 'ID' column is not null and not empty
        if pd.notnull(row['ID']) and row['ID']:
            # Split the IDs in the 'ID' column
            ids = row['ID'].split()

            # Store sentiment for each ID in the dictionary
            for news_id in ids:
                id_sentiments[news_id] = row['Sentiment']

    # Create a new dataframe to store the results
    result_df = behaviors_df.copy()

    # Initialize columns for sentiments
    result_df['Negative'] = 0
    result_df['Positive'] = 0
    result_df['Neutral'] = 0

    # Iterate through rows in the behaviors dataframe
    for index, row in result_df.iterrows():
        # Check if the 'ID' column is not null and not empty
        if pd.notnull(row['ID']) and row['ID']:
            # Split the IDs in the 'ID' column
            ids = row['ID'].split()

            # Count sentiments for each ID and update the result dataframe
            for behavior_id in ids:
                sentiment = id_sentiments.get(behavior_id, 'Neutral')  # Default to 'Neutral'
                result_df.at[index, sentiment] += 1

    return result_df


In [18]:
user_profile_sentiments = analyze_sentiments(news, interactions)
user_profile_sentiments.head()

,User,ID,Interactions_emb,Negative,Positive,Neutral
0,U244,N17157 N38621 N35022 N50578 N264 N9120 N23907 ...,"[-0.005149974951877837, -0.013250857458654631,...",14,11,4
1,U68369,N19381 N54536,"[0.0025621717686590273, 0.004183989018201828, ...",2,0,0
2,U50236,N4020 N44292 N50292 N40772 N57737 N33969 N4054...,"[-0.010138329240492436, -0.01179651383115145, ...",11,26,21
3,U77060,N23105 N41375,"[-0.005568941123783588, -0.025914330035448074,...",1,1,0
4,U5596,N459 N56253 N62931 N55846 N29849 N45729 N62834...,"[-0.012533644353970886, -0.011675744312297967,...",11,23,11


In [21]:
def analyze_sentiments_recommended(news_df, behaviors_df, n):
    # Create a dictionary to store sentiments for each ID
    id_sentiments = {}

    # Iterate through rows in the news dataframe
    for index, row in news_df.iterrows():
        # Split the IDs in the 'Content' column
        ids = row['ID'].split()

        # Store sentiment for each ID in the dictionary
        for news_id in ids:
            id_sentiments[news_id] = row['Sentiment']

    # Create a new dataframe to store the results
    result_df = behaviors_df.copy()

    # Initialize columns for recommended sentiments
    result_df['Negative_recommended'] = 0
    result_df['Positive_recommended'] = 0
    result_df['Neutral_recommended'] = 0

    # Iterate through rows in the behaviors dataframe
    for index, row in result_df.iterrows():
        user_id = row['User']
        recommendations_file = f'content_recommendations/{user_id}_content.csv'

        # Check if the recommendations file exists
        if os.path.exists(recommendations_file):
            # Read the content recommendations file
            recommendations_df = pd.read_csv(recommendations_file)

            # Take the top n recommendations
            top_n_recommendations = recommendations_df.head(n)

            # Count sentiments for each recommended ID and update the result dataframe
            for recommended_id in top_n_recommendations['ID']:
                sentiment = id_sentiments.get(recommended_id, 'Neutral')  # Default to 'Neutral'
                result_df.at[index, f'{sentiment}_recommended'] += 1

    return result_df


In [22]:
n_recommendations = 10

result_df = analyze_sentiments_recommended(news, user_profile_sentiments, n_recommendations)
result_df.head()

,User,ID,Interactions_emb,Negative,Positive,Neutral,Negative_recommended,Positive_recommended,Neutral_recommended
0,U244,N17157 N38621 N35022 N50578 N264 N9120 N23907 ...,"[-0.005149974951877837, -0.013250857458654631,...",14,11,4,2,5,3
1,U68369,N19381 N54536,"[0.0025621717686590273, 0.004183989018201828, ...",2,0,0,9,1,0
2,U50236,N4020 N44292 N50292 N40772 N57737 N33969 N4054...,"[-0.010138329240492436, -0.01179651383115145, ...",11,26,21,3,4,3
3,U77060,N23105 N41375,"[-0.005568941123783588, -0.025914330035448074,...",1,1,0,2,4,4
4,U5596,N459 N56253 N62931 N55846 N29849 N45729 N62834...,"[-0.012533644353970886, -0.011675744312297967,...",11,23,11,4,1,5


In [24]:
# Calculate Pearson correlation coefficient and p-value for each pair
correlations = {}

# Iterate over each pair of columns
for column in ['Negative', 'Positive', 'Neutral']:
    correlation_coefficient, p_value = pearsonr(result_df[column], result_df[column+'_recommended'])
    correlations[column] = {
        'correlation_coefficient': correlation_coefficient,
        'p_value': p_value
    }

# Display the results
for column, values in correlations.items():
    print(f"Pair: {column} and {column}_recommended")
    print(f"Pearson Correlation Coefficient: {values['correlation_coefficient']}")
    print(f"P-value: {values['p_value']}")
    print()


Pair: Negative and Negative_recommended
Pearson Correlation Coefficient: 0.09720811193623459
P-value: 3.963276020477676e-105

Pair: Positive and Positive_recommended
Pearson Correlation Coefficient: 0.10543972828482541
P-value: 1.9239265607044678e-123

Pair: Neutral and Neutral_recommended
Pearson Correlation Coefficient: 0.14543019624246156
P-value: 3.618313345682346e-234

